In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.5


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/workspaces/Regression_ML_EndtoEnd/.venv/bin/python
3.0.5
/workspaces/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [5]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

In [6]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/workspaces/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/workspaces/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [7]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/workspaces/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/10/02 12:33:30 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-10-02 12:33:30,210] A new study created in memory with name: no-name-dfd2ce1d-1a3d-49eb-aa6c-82231af3a70b


[I 2025-10-02 12:34:17,624] Trial 0 finished with value: 71415.53305533597 and parameters: {'n_estimators': 305, 'max_depth': 10, 'learning_rate': 0.03407501606727369, 'subsample': 0.8602877885190545, 'colsample_bytree': 0.5892187410236365, 'min_child_weight': 5, 'gamma': 1.4558180216602856, 'reg_alpha': 2.538093154267891, 'reg_lambda': 1.3612045179653684e-08}. Best is trial 0 with value: 71415.53305533597.
[I 2025-10-02 12:35:41,570] Trial 1 finished with value: 75106.61915531826 and parameters: {'n_estimators': 788, 'max_depth': 9, 'learning_rate': 0.1915756271874383, 'subsample': 0.7601530688482399, 'colsample_bytree': 0.7025958286567734, 'min_child_weight': 5, 'gamma': 2.7229294239089237, 'reg_alpha': 2.6943146061903858, 'reg_lambda': 2.3265713889444713e-08}. Best is trial 0 with value: 71415.53305533597.
[I 2025-10-02 12:37:22,878] Trial 2 finished with value: 72601.35246149 and parameters: {'n_estimators': 934, 'max_depth': 9, 'learning_rate': 0.06882219424675885, 'subsample': 0.

Best params: {'n_estimators': 483, 'max_depth': 6, 'learning_rate': 0.06681540831190468, 'subsample': 0.949719208554441, 'colsample_bytree': 0.5041070772805798, 'min_child_weight': 7, 'gamma': 0.23574481400436054, 'reg_alpha': 3.1548471850647093e-06, 'reg_lambda': 0.016256976176657015}


In [ ]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 32971.394497178015
RMSE: 69734.64355295326
R²: 0.962419963126363


/workspaces/Regression_ML_EndtoEnd/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [12:46:42] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/10/02 12:46:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


: 